In [29]:
from math import log2,abs

In [23]:
def cleanDataset(dataset):
    for row in dataset:
        for col in row:
            if col == '?' or col == '':
                print(row)
                dataset.remove(row)

def createDataset(filename):
    with open(filename,'r') as csvfile:
        dataset = [row.strip().split(', ') for row in csvfile]
        dataset = [[int(i) if i.isdigit() else i for i in row] for row in dataset]
        # cleanDataset(dataset)
        del(dataset[-1])
        labels=['age','workclass','fnlwgt','education','education-num',
                'marital-status','occupation',
                 'relationship','race','sex','capital-gain','capital-loss','hours-per-week',
                 'native-country']
        labelType = ['continuous', 'uncontinuous', 'continuous',
                      'uncontinuous',
                      'continuous', 'uncontinuous',
                      'uncontinuous', 'uncontinuous', 'uncontinuous',
                      'uncontinuous', 'continuous', 'continuous',
                      'continuous', 'uncontinuous']
        return dataset,labels,labelType
        
dataset,labels,labelType = createDataset('adult/adult.data')



In [30]:
def calculateEnt(dataset):
    classCount  = {}
    n = len(dataset)
    for vec in dataset:
        classification = vec[-1]
        if classification not in classCount.keys():
            classCount[classification] = 0
        classCount[classification] += 1
    ent = 0.0
    for key in classCount:
        p = classCount[key]/n
        ent += -1*(p*log2(p))
    return ent

0.7963839552022132

In [33]:
def splitDataset(dataset,labelIdx,value):
    newDataset = []
    for vec in dataset:
        if vec[labelIdx] == value:
            tmp = vec[:labelIdx]
            tmp.extend(vec[labelIdx+1:])
            newDataset.append(tmp)
    return newDataset

def splitContinuousDataset(dataset,labelIdx,value):
    biggerDataset = []
    smallerDataset = []
    for vec in dataset:
        tmp = vec[:labelIdx]
        tmp.extend(vec[labelIdx+1:])
        if vec[labelIdx] > value:
            biggerDataset.append(tmp)
        else:
            smallerDataset.append(tmp)
    return biggerDataset,smallerDataset

In [37]:
def calGainRatioUnContinuous(dataset,labelIdx,ent):
    #获取该特征的类别
    featureList = [vec[labelIdx] for vec in dataset]
    uniqueFeature = set(featureList)
    #条件熵
    entc = 0.0
    #特征分裂信息度量
    iv = 0.0
    #计算条件熵和特征分裂信息度量
    for val in uniqueFeature:
        subDataset = splitDataset(dataset,labelIdx,val)
        D = len(subDataset) / len(dataset)
        iv += -1*(D*log2(D))
        entc += D*calculateEnt(subDataset)
    gain = ent - entc
    #全是一个类别split = 1
    if iv == 0 :
        iv = 1
    #计算gainRatio
    gainRadio = gain / iv
    return gainRadio


In [ ]:
def calGainRatioContinuous(dataset,labelIdx,ent):
    #获取该连续特征的各种值
    valueList = [vec[labelIdx] for vec in dataset]
    valueList = set(featureList)
    sortedValue = sorted(valueList)
    #获取n-1个分割点
    splitPoint = []
    for i in range(len(sortedValue)-1):
        splitPoint.append((sortedValue[i]+sortedValue[i+1])/2.0)
    #寻找最优分割点(信息增益最大)
    bestGainRatio = 0.0
    bestGain = 0.0
    bestSplitPoint = 0.0
    for i in range(len(splitPoint)):
        entc = 0.0
        iv = 0.0
        biggerDataset,smallerDataset = splitContinuousDataset(dataset,labelIdx,splitPoint[i])
        Db = len(biggerDataset) / len(dataset)
        Ds = len(smallerDataset) / len(dataset)
        entc += Db*calculateEnt(biggerDataset)
        entc += Ds*calculateEnt(smallerDataset)
        gain = ent - entc
        if gain > bestGain:
            bestGain = gain
            iv += -1*(Db*log2(Db))
            iv += -1*(Ds*log2(Ds))
            bestSplitPoint = splitPoint[i]
            #修正信息增益
            bestGain-=log2(len(valueList)-1)/abs(len(dataset))
            #计算最优信息增益率
            bestGainRatio = bestGain / iv
    return bestGainRatio,bestSplitPoint


In [ ]:
def chooseBestSplit(dataset,labels):
    

2
